# Morphologic Transformations Visualizations 🔲

This notebook contains algorithms for visualizing some basic morphologic transformations such as erosion, dilation, opening, and closing. 

It serves the purpose of offering a better understanding in the process of exploring Image Processing. 📚

In [2]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Dilation and Erosion

In [3]:
input_img = cv.imread("./img/mon1thr1_bw.bmp", cv.IMREAD_GRAYSCALE)
# cv.imshow("input", input_img)
# cv.waitKey(0)                    # press on any key to close the image
# cv.destroyWindow("input") 

## Out-of-box behavior

In [3]:
cv.imshow("source", input_img)

default_dilated = cv.dilate(input_img, None, iterations = 1)

default_eroded = cv.erode(input_img, None, iterations = 1)

cv.imshow("default dilation", default_dilated)
cv.imshow("default eroded", default_eroded)

cv.waitKey(0)
cv.destroyWindow("source")
cv.destroyWindow("default dilation")
cv.destroyWindow("default eroded")

*Question: Why is it reversed?*

## Implementation and Visualization

In [5]:
def getOffsets(structuring_elem):
    elem_matrix, origin = structuring_elem
    x_origin, y_origin = origin

    (x_coord, y_coord) = np.where(elem_matrix == 1)    
    x_coord = x_coord - x_origin    
    y_coord = y_coord - y_origin
    
    offsets = list(zip(x_coord, y_coord))
    return offsets
    
def inImage(curr_i, curr_j, rows, cols):
    if curr_i > 0 and curr_i < rows:
        if curr_j > 0 and curr_j < cols:
            return True
    return False

def customErosion(src, structuring_elem, iterations, bg_col):
    """
    Performs erosion on src and returns the eroded image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(dst)
    structuring_elem_offsets = getOffsets(structuring_elem)

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    all_neigbors_are_object = True
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            curr_neigh = src[i + offset_i, j + offset_j] if iter == 0 else dst[i + offset_i, j + offset_j]
                            if curr_neigh == 255:
                                all_neigbors_are_object = False
                                break
                    if all_neigbors_are_object:
                        aux[i, j] = 0 # color the origin as object
                    else:
                        aux[i, j] = 255
        dst = np.copy(aux)

    return dst
                            

def customDilation(src, structuring_elem, iterations, bg_col):
    """
    Performs dilation on src and returns the dilated image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(dst)
    structuring_elem_offsets = getOffsets(structuring_elem)

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            aux[i + offset_i, j + offset_j] = 0
        dst = np.copy(aux)
    return dst


def main() :
    structuring_elem = (np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]), (1, 1))
    # eroded = customErosion(input_img, structuring_elem, 1, 1)
    # dilated = customDilation(input_img, structuring_elem, 1, 1)
    # cv.imshow("source", input_img)
    # cv.imshow("eroded", eroded)
    # cv.imshow("dilated", dilated)

    # cv.waitKey(0)
    # cv.destroyWindow("eroded")
    # cv.destroyWindow("dilated")
    # cv.destroyWindow("source")


if __name__ == '__main__':
    main()

### Creating animations to show the workings of erosion and dilation
Ideas:
- show intermmediary steps (export pictures and combine them in a small video)
    - try to zoom in somehow
- overlap the images in a color image

In [13]:
import os
import imageio

def overlapStructElemAndSave(src, i, j, offsets, save_path):
    rows, cols = src.shape
    to_return= np.full((rows, cols, 3), 255, np.uint8) # White bg

    # Leave object pixels black
    to_return[:, :, 0] = src
    to_return[:, :, 1] = src
    to_return[:, :, 2] = src

    for offset_i, offset_j in offsets:
        index_i = i + offset_i
        index_j = j + offset_j
        if(to_return[index_i, index_j, 1] == 0): # intersection between struct elem and object (dark green)
            to_return[i + offset_i, j + offset_j, 1] = 99
            to_return[i + offset_i, j + offset_j, 2] = 18
        else: # structuring element outside object (green)
            to_return[i + offset_i, j + offset_j, 0] = 0
            to_return[i + offset_i, j + offset_j, 2] = 0

    # highlight object center (yellow)
    to_return[i, j, 1] = 255
    to_return[i, j, 2] = 255

    cv.imwrite(save_path, to_return)

def customDilationWithVisualization(src, structuring_elem, iterations, bg_col):
    """
    Performs dilation on src and returns the dilated image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(src)
    structuring_elem_offsets = getOffsets(structuring_elem)

    os_path = "./outDilation/img"
    count_img = 0

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    last_i, last_j = i,j
                    path = os_path + str(count_img) + ".bmp"
                    overlapStructElemAndSave(aux, i, j, structuring_elem_offsets, path)
                    count_img += 1

                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            aux[i + offset_i, j + offset_j] = 0

        path = os_path + str(count_img) + ".bmp"
        overlapStructElemAndSave(aux, last_i, last_j, structuring_elem_offsets, path)
                    
        dst = np.copy(aux)
    return dst

def customErosionWithVisualization(src, structuring_elem, iterations, bg_col):
    """
    Performs erosion on src and returns the eroded image.
    Parameters:
    - src - 2D numpy array of uintc
    - structuring element - tuple (2D numpy array of uintc filled with 1s or 0s, origin_coordinates)
    """
    rows, cols = src.shape
    if bg_col == 0 :
        dst = np.zeros((rows, cols), np.uint8)
    else:
        dst = np.full((rows, cols), 255, np.uint8)

    aux = np.copy(dst)
    structuring_elem_offsets = getOffsets(structuring_elem)

    os_path = "./outDilation/img"
    count_img = 0

    for iter in range(iterations):
        for i in range(rows):
            for j in range(cols):
                curr_pixel = src[i, j] if iter == 0 else dst[i, j]
                if curr_pixel == 0:
                    last_i, last_j = i, j
                    path = os_path + str(count_img) + ".bmp"
                    overlapStructElemAndSave(aux, i, j, structuring_elem_offsets, path)
                    count_img += 1
                    all_neigbors_are_object = True
                    for (offset_i, offset_j) in structuring_elem_offsets:
                        if inImage(i + offset_i, j + offset_j, rows, cols):
                            curr_neigh = src[i + offset_i, j + offset_j] if iter == 0 else dst[i + offset_i, j + offset_j]
                            if curr_neigh == 255:
                                all_neigbors_are_object = False
                                break
                    if all_neigbors_are_object:
                        aux[i, j] = 0 # color the origin as object
                    else:
                        aux[i, j] = 255
        dst = np.copy(aux)

    path = os_path + str(count_img) + ".bmp"
    overlapStructElemAndSave(aux, last_i, last_j, structuring_elem_offsets, path)         
                    
    return dst


def videoFromImages(src_dir, nr_images, out_name):
    images = []
    file = src_dir + "img"
    for count in range(nr_images):
        filepath = file + str(count) + ".bmp"
        img = cv.imread(filepath)
        print(filepath)
        images.append(img)

    imageio.mimsave(os.path.join(src_dir, out_name), images, duration=40)
    return

def main() :
    input_img = cv.imread("./img/lines.bmp", cv.IMREAD_GRAYSCALE)
    structuring_elem = (np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]]), (1, 1))
    dilated = customDilationWithVisualization(input_img, structuring_elem, 1, 1)
    videoFromImages("./outDilation/", 197, "dilation_gif.gif")
    
    input_erosion = cv.imread("./img/circle.bmp", cv.IMREAD_GRAYSCALE)
    eroded = customErosionWithVisualization(input_erosion, structuring_elem, 1, 1)
    # cv.imshow("source", input_img)
    # cv.imshow("dilated", dilated)

    # cv.waitKey(0)
    # cv.destroyWindow("dilated")
    # cv.destroyWindow("source")


if __name__ == '__main__':
    main()

./outDilation/img0.bmp
./outDilation/img1.bmp
./outDilation/img2.bmp
./outDilation/img3.bmp
./outDilation/img4.bmp
./outDilation/img5.bmp
./outDilation/img6.bmp
./outDilation/img7.bmp
./outDilation/img8.bmp
./outDilation/img9.bmp
./outDilation/img10.bmp
./outDilation/img11.bmp
./outDilation/img12.bmp
./outDilation/img13.bmp
./outDilation/img14.bmp
./outDilation/img15.bmp
./outDilation/img16.bmp
./outDilation/img17.bmp
./outDilation/img18.bmp
./outDilation/img19.bmp
./outDilation/img20.bmp
./outDilation/img21.bmp
./outDilation/img22.bmp
./outDilation/img23.bmp
./outDilation/img24.bmp
./outDilation/img25.bmp
./outDilation/img26.bmp
./outDilation/img27.bmp
./outDilation/img28.bmp
./outDilation/img29.bmp
./outDilation/img30.bmp
./outDilation/img31.bmp
./outDilation/img32.bmp
./outDilation/img33.bmp
./outDilation/img34.bmp
./outDilation/img35.bmp
./outDilation/img36.bmp
./outDilation/img37.bmp
./outDilation/img38.bmp
./outDilation/img39.bmp
./outDilation/img40.bmp
./outDilation/img41.bmp
./

NameError: name 'input_erosion' is not defined